 ## Reading and Writing Data in Text Format
 
 pandas features a number of functions for reading tabular data as a DataFrame object. 
 
 **Type Inference** means you don't have to specify which columns are numeric, integre, boolean or string. 

In [50]:
%cd "C:\Users\sonya\Documents\Python for Data Analysis\data\ch06"
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

C:\Users\sonya\Documents\Python for Data Analysis\data\ch06


In [8]:
#Preview data file using unix type command to print the raw contents of the file

!type ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [9]:
df=pd.read_csv('ex1.csv') #reads comma delimited by default. To specify delimiter, use sep = ',' option
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


A file will not always have a header row. For example

In [10]:
!Type ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


To read this in, allow pandas to assign default column names by specifying `header=None` option or specify names yourself using `name=` option. If we omit `header=None` option, pandas will automatically take the first row as its headers.

In [11]:
pd.read_csv('ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [12]:
pd.read_csv('ex2.csv')

,1,2,3,4,hello
0,5,6,7,8,world
1,9,10,11,12,foo


In [13]:
pd.read_csv('ex2.csv',names=['a','b','c','d','message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


To make a column an index, we can use `index_col` argument and indicate we want the column at index # or name of the column. If we want a hierarchical index from multiple columns, we can pass a list of column numbers of names through `index_col`

In [14]:
# Make 'message' our index for the df

names=['a','b','c','d','message']

pd.read_csv('ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [15]:
!Type csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [16]:
parsed = pd.read_csv('csv_mindex.csv',index_col=['key1','key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In some cases, we need to use whitespace as our delimiter. In such cases, we can use `read_table` method and ad `\s+` as our separator for space via `sep=`.

In [17]:
!Type ex3.txt

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


In [20]:
result = pd.read_table('ex3.txt', sep ='\s+')
result

# There's no column name in column 1 so read_table infers that the first column should be the DatFrame's index

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


#### Additional Options

Skip rows, handle missing data, etc.

In [21]:
!Type ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [22]:
pd.read_csv('ex4.csv',skiprows=[0,2,3])  # Skip rows

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


For missing values, pandas uses a set of commonly occurring **sentinels** like NA, -1.#IND, and NULL.

**Sentinels** are objects with special meanings. They can be thought of as singletons, but they service the need of having 'special' values in your code, that have special meanings

In [23]:
!Type ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [25]:
result = pd.read_csv('ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [26]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [28]:
result = pd.read_csv('ex5.csv',na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Different NA sentinels can be specified for each column in a dict:

In [29]:
sentinels = {'message': ['foo','NA'],'something':['two']}

pd.read_csv('ex5.csv',na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### Reading Text Files in Pieces 

In [31]:
result = pd.read_csv('ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
5,1.817480,0.742273,0.419395,-2.251035,Q
6,-0.776764,0.935518,-0.332872,-1.875641,U
7,-0.913135,1.530624,-0.572657,0.477252,K
8,0.358480,-0.497572,-0.367016,0.507702,S
9,-1.740877,-1.160417,-1.637830,2.172201,G


To avoid reading the whole file, read out only a small number of rows using `nrows=`

In [34]:
pd.read_csv('ex6.csv', nrows=10) # First 10 rows

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
5,1.817480,0.742273,0.419395,-2.251035,Q
6,-0.776764,0.935518,-0.332872,-1.875641,U
7,-0.913135,1.530624,-0.572657,0.477252,K
8,0.358480,-0.497572,-0.367016,0.507702,S
9,-1.740877,-1.160417,-1.637830,2.172201,G


In [35]:
# To read out a file in pieces, specify a chunksize as number of rows

chunker = pd.read_csv('ex6.csv', chunksize = 1000)
chunker

In [36]:
# TextParser object returned by read_csv allows us to iterate over the parts of the file according to chunksize
# For ex, we can iterate over ex6 summing the value counts in the key column

tot=Series([])
for piece in chunker:
    tot=tot.add(piece['key'].value_counts(),fill_value=0)

tot = order(ascending=False)

NameError: name 'order' is not defined

### Writing Data Out to Text Format

Data can also be exported to delimited format using `to_csv` method 

In [39]:
data=pd.read_csv('ex5.csv')
data

data.to_csv('out.csv')

!Type out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [41]:
data.to_csv('out2.csv', sep='|')
!Type out2.csv

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


By default, missing values appear as empty strings in the output. We might want to denote them by some other sentinel value using `na_rep`:

In [43]:
data.to_csv('out3.csv',na_rep='N/A')
!Type out3.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,N/A
1,two,5,6,N/A,8,world
2,three,9,10,11.0,12,foo


By default, both row and column labels are written. This can be disabled using `header=False and index=False`

In [45]:
data.to_csv('out4.csv',index=False,header=False)
!Type out4.csv

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


We can also only write a subset of the column and in an order of our choosing

In [48]:
data.to_csv('out5.csv',index=False, columns=['a','b','c'])
!Type out5.csv

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [53]:
# Also works on series

dates = pd.date_range('1/1/2000',periods=7)
print dates

ts=Series(np.arange(7),index=dates)
print ts

ts.to_csv('tseries.csv')
!Type tseries.csv

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07'],
              dtype='datetime64[ns]', freq='D')
2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int32
2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


In [55]:
# Read a CSV version of series (with no header, first column as index) with read_csv or from_csv

Series.from_csv('tseries.csv', parse_dates=True)

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
dtype: int64

### Manually Working with Delimited Formats

Most forms of tabular data can be loaded using pandas.read_table. In some cases, however, some manual processing may be necessary.

In [56]:
!Type ex7.csv

"a","b","c"
"1","2","3"
"1","2","3","4"


In [57]:
import csv

f = open('ex7.csv')
reader= csv.reader(f)

iterating through the reader like a file yields tuples of values in each like with any quote characters removed:

In [58]:
for line in reader:
    print line

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3', '4']


From there, it's up to us to do the wrangling necessary to put the data in the form we need it.

In [59]:
lines = list(csv.reader(open('ex7.csv')))

header, values = lines[0], lines[1:] #values are in Row 1 onwards and header is in Row 0

data_dict= {h: v for h, v in zip(header,zip(*values))}

data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

CSV files come in many different formats. Defining a new format with a set delimiter, string quoting convention or line terminator is done by defining a simple subclass of `csv.Dialect`:

In [62]:
class my_dialect(csv.Dialect):
    lineterminator='\n'
    delimiter=';'
    quotechar='"'
    quoting=csv.QUOTE_MINIMAL

reader=csv.reader(f,dialect=my_dialect)

To write delimited files manually, we can use csv.writer. It accepts an open writable file object and same dialect/format options as csv.reader

In [67]:
with open('mydata.csv', 'w') as f:
    writer=csv.writer(f, dialect=my_dialect)
    writer.writerow(('one','two','three'))
    writer.writerow(('1','2','3'))
    writer.writerow(('4','5','6'))
    writer.writerow(('7','8','9'))
    
readout = pd.read_csv('mydata.csv',sep=';')
readout

,one,two,three
0,1,2,3
1,4,5,6
2,7,8,9


### JSON Data

**JSON ( Javascript Object Notation)** is a standard format for sending data by HTTP request between web browsers and other apps. It's a much more flexible data format than a tabular text form like CSV.

In [71]:
obj="""
{"name":"Wes",
 "places_lived": ["United States","Spain","Germany"],
 "pets": null,
 "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
              {"name": "Katie", "age": 33, "pet": "Cisco"}]
}
"""

All the keys in an object must be strings. There are several libraries for reading and writing JSON data such as Python's built in `json` standard library. To convert a JSON string to Python form, use `json.loads`:

In [88]:
import json

result=json.loads(obj)

result

{u'name': u'Wes',
 u'pets': None,
 u'places_lived': [u'United States', u'Spain', u'Germany'],
 u'siblings': [{u'age': 25, u'name': u'Scott', u'pet': u'Zuko'},
  {u'age': 33, u'name': u'Katie', u'pet': u'Cisco'}]}

`json.dumps`converts a python object back to JSON

In [75]:
asjson=json.dumps(result)
asjson

'{"siblings": [{"pet": "Zuko", "age": 25, "name": "Scott"}, {"pet": "Cisco", "age": 33, "name": "Katie"}], "name": "Wes", "pets": null, "places_lived": ["United States", "Spain", "Germany"]}'

You can pass a list of JSON objects to the DataFrame constructor and select a subset of the data fields

In [89]:
siblings = DataFrame(result['siblings'], columns=['name','age'])

siblings


,name,age
0,Scott,25
1,Katie,33


###  Binary Data Formats

The easiest way to store data efficiently in binary format is using Python's built-in `pickle` serialization. There is also a `save` method which write data to disk as a pickle:

In [90]:
frame=pd.read_csv('ex1.csv')

frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [98]:
import cPickle as pickle

#Store data / Serialize
with open('frame_pickle.pickle', 'wb') as handle:
    pickle.dump(frame, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('frame_pickle.pickle', 'rb') as handle:
    unserialized_data = pickle.load(handle)
    
print(frame == unserialized_data)

      a     b     c     d  message
0  True  True  True  True     True
1  True  True  True  True     True
2  True  True  True  True     True


### Reading Excel Files

pandas supports reading tabular data stored in Excel. 

More info: https://automatetheboringstuff.com/chapter12/

In [103]:
import openpyxl as excel

xls_file = pd.ExcelFile('data.xlsx')

table=xls_file.parse('Sheet1')

table

,Testing,1
0,1,3.0
1,2,3.0
2,3,NaN
3,4,3.0


### Interacting with HTML and Web API's

Many website have public API's providing data feeds via JSON or some other format. 

In [112]:
import requests

url = 'https://api.github.com/repos/pydata/pandas/milestones/28/labels'

resp=requests.get(url)
resp

data = resp.json()  # json returns a dictionary containing JSON parsed into native Python objects
print data

issue_labels=DataFrame(data)
print issue_labels

[{u'url': u'https://api.github.com/repos/pandas-dev/pandas/labels/Bug', u'color': u'e10c02', u'default': False, u'id': 76811, u'name': u'Bug'}, {u'url': u'https://api.github.com/repos/pandas-dev/pandas/labels/Enhancement', u'color': u'4E9A06', u'default': False, u'id': 76812, u'name': u'Enhancement'}, {u'url': u'https://api.github.com/repos/pandas-dev/pandas/labels/Refactor', u'color': u'FCE94F', u'default': False, u'id': 127681, u'name': u'Refactor'}, {u'url': u'https://api.github.com/repos/pandas-dev/pandas/labels/Build', u'color': u'75507B', u'default': False, u'id': 129350, u'name': u'Build'}, {u'url': u'https://api.github.com/repos/pandas-dev/pandas/labels/Docs', u'color': u'3465A4', u'default': False, u'id': 134699, u'name': u'Docs'}, {u'url': u'https://api.github.com/repos/pandas-dev/pandas/labels/Groupby', u'color': u'729FCF', u'default': False, u'id': 233160, u'name': u'Groupby'}, {u'url': u'https://api.github.com/repos/pandas-dev/pandas/labels/Data%20IO', u'color': u'06909A',

In [114]:
issue_labels.iloc[7]

color                                                 8AE234
default                                                False
id                                                   2413328
name                                           Visualization
url        https://api.github.com/repos/pandas-dev/pandas...
Name: 7, dtype: object

### Databases

In many applications data rarely comes from text files, that being a fairly inefficient way to store large amounts of data. SQL-based relational databases (MYSQL, SQL Server, PostGresSQL) are in wide use. Loading data from SQL into a DataFrame is fairly easy and pandas has some functions to simplify the process. 

In [ ]:
import sqlite3

query ="""
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
c REAL, d INTEGER);"""

con=sqlite.3.connect(':memory:')
con.execute(query)
con.commit()

data=[('Atlanta','Georgia', 1.25, 6),
      ('Tallahassee','Florida', 2.6,3),
      ('Sacramento','California',1.7,5)]

stmt= "INSERT INTO test VALUES(?,?,?,?)"

con.executemany(stmt, data)
con.commit()